In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='collected_data.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('collected_data.zip')
f.keys()
!unzip collected_data.zip


Streaming output truncated to the last 5000 lines.
  inflating: collected_data/0/0_390.jpg  
  inflating: collected_data/0/0_391.jpg  
  inflating: collected_data/0/0_392.jpg  
  inflating: collected_data/0/0_393.jpg  
  inflating: collected_data/0/0_394.jpg  
  inflating: collected_data/0/0_395.jpg  
  inflating: collected_data/0/0_396.jpg  
  inflating: collected_data/0/0_397.jpg  
  inflating: collected_data/0/0_398.jpg  
  inflating: collected_data/0/0_399.jpg  
  inflating: collected_data/0/0_4.jpg  
  inflating: collected_data/0/0_40.jpg  
  inflating: collected_data/0/0_400.jpg  
  inflating: collected_data/0/0_401.jpg  
  inflating: collected_data/0/0_402.jpg  
  inflating: collected_data/0/0_403.jpg  
  inflating: collected_data/0/0_404.jpg  
  inflating: collected_data/0/0_405.jpg  
  inflating: collected_data/0/0_406.jpg  
  inflating: collected_data/0/0_407.jpg  
  inflating: collected_data/0/0_408.jpg  
  inflating: collected_data/0/0_409.jpg  
  inflating: collected_data/

In [4]:
gpus = tf.config.list_logical_devices('GPU')
stg=tf.distribute.MirroredStrategy(gpus)

In [5]:
import cv2
import os
from tqdm import tqdm

folder_dir = 'collected_data'
SIZE = 224
DOWNSAMPLE_RATIO = 4
JPEG_QUALITY = 100

total_files = sum(len(files) for _, _, files in os.walk(folder_dir))

with tqdm(total=total_files, desc="Processing Images") as pbar:
    for folder in os.listdir(folder_dir):
        for file in os.listdir(os.path.join(folder_dir, folder)):
                image_path = os.path.join(folder_dir, folder, file)
                img = cv2.imread(image_path)
                img_resized = cv2.resize(img, (SIZE,SIZE))
                cv2.imwrite(image_path, img_resized)
                pbar.update(1)

Processing Images: 100%|██████████| 5314/5314 [01:14<00:00, 71.42it/s]


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [7]:
batch_size = 32
picture_size = (224, 224)
train_set = tf.keras.utils.image_dataset_from_directory(
    directory=folder_dir,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed = 22
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory=folder_dir,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed = 22
)

Found 5314 files belonging to 10 classes.
Using 4252 files for training.
Found 5314 files belonging to 10 classes.
Using 1062 files for validation.


In [8]:
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomTranslation, RandomFlip, RandomZoom
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import shutil
import random
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

no_of_classes = 10

# Define the Xception base model
base_model = Xception(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))

# Create three instances of the base model with different inputs
input_1 = base_model.input
output_1 = base_model.output
input_2 = base_model.input
output_2 = base_model.output
input_3 = base_model.input
output_3 = base_model.output

# Add global average pooling layer and dense layers for each model
output_1 = GlobalAveragePooling2D()(output_1)
output_1 = Dense(512, activation='relu')(output_1)
output_1 = Dropout(0.5)(output_1)
output_1 = Dense(32, activation='relu')(output_1)
output_1 = Dense(no_of_classes, activation='softmax')(output_1)

output_2 = GlobalAveragePooling2D()(output_2)
output_2 = Dense(512, activation='relu')(output_2)
output_2 = Dropout(0.5)(output_2)
output_2 = Dense(32, activation='relu')(output_2)
output_2 = Dense(no_of_classes, activation='softmax')(output_2)

output_3 = GlobalAveragePooling2D()(output_3)
output_3 = Dense(512, activation='relu')(output_3)
output_3 = Dropout(0.5)(output_3)
output_3 = Dense(32, activation='relu')(output_3)
output_3 = Dense(no_of_classes, activation='softmax')(output_3)

# Create three separate models
model_1 = Model(inputs=input_1, outputs=output_1)
model_2 = Model(inputs=input_2, outputs=output_2)
model_3 = Model(inputs=input_3, outputs=output_3)

# Compile the models
model_1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_2.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_3.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

83683744/83683744 [==============================] - 1s 0us/step


In [9]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.1,
                                 patience=2,
                                 verbose=1)
early_stopping = EarlyStopping(monitor='val_loss',
                                patience=2,
                                verbose=1)

callbacks = [lr_scheduler, early_stopping]

In [10]:
# Train model 1
model_1.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

# Train model 2
model_2.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

# Train model 3
model_3.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

Epoch 1/10
133/133 [==============================] - 126s 580ms/step - loss: 0.6968 - accuracy: 0.7843 - val_loss: 63.1782 - val_accuracy: 0.6375 - lr: 0.0010
Epoch 2/10
133/133 [==============================] - 70s 522ms/step - loss: 0.2024 - accuracy: 0.9534 - val_loss: 2.5349 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 3/10
133/133 [==============================] - 70s 523ms/step - loss: 0.1021 - accuracy: 0.9791 - val_loss: 0.0464 - val_accuracy: 0.9887 - lr: 0.0010
Epoch 4/10
133/133 [==============================] - 68s 513ms/step - loss: 0.0643 - accuracy: 0.9854 - val_loss: 0.1946 - val_accuracy: 0.9661 - lr: 0.0010
Epoch 5/10
133/133 [==============================] - ETA: 0s - loss: 0.0793 - accuracy: 0.9819
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
133/133 [==============================] - 70s 522ms/step - loss: 0.0793 - accuracy: 0.9819 - val_loss: 1.5124 - val_accuracy: 0.9002 - lr: 0.0010
Epoch 5: early stopping
Epoch 1/10
133/133 [==

In [11]:
ensemble_predictions = []
num_models = 3
test_data_dir = '/content/collected_data'
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)
# Generate predictions for each model and collect them in a list
for model in [model_1, model_2, model_3]:
    predictions = model.predict_generator(test_generator)
    ensemble_predictions.append(predictions)

# Compute the average predictions of the ensemble
ensemble_predictions = np.mean(ensemble_predictions, axis=0)

# Calculate the accuracy of the ensemble predictions
ensemble_acc = np.mean(np.argmax(ensemble_predictions, axis=1) == test_generator.classes)

print('Ensemble accuracy:', ensemble_acc)

Found 5314 images belonging to 10 classes.


<ipython-input-11-4f78912304d7>:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator)


Ensemble accuracy: 0.8959352653368461
